In [ ]:
import numpy as np
from astropy import units as u
from specutils import Spectrum1D, SpectralRegion

from jdaviz import Specviz

In [ ]:
wave = np.linspace(1, 5, 100) * u.um
flux = np.random.random(wave.shape) * u.nJy
sp = Spectrum1D(flux=flux, spectral_axis=wave)

In [ ]:
specviz = Specviz()

In [ ]:
specviz.load_data(sp, data_label="Spectrum")

In [ ]:
specviz.show()

In [ ]:
plg_subset = specviz.plugins["Subset Tools"]

In [ ]:
# This kinda works because Export simplifies it to
#     lower_bound upper_bound
#     1.5 3.0
#     3.5 4.5
#
# Same output but original values lost.
spreg_1 = SpectralRegion(1.5 * u.um, 4.5 * u.um)
spreg_2 = SpectralRegion(3 * u.um, 3.5 * u.um)
plg_subset.import_region([spreg_1, spreg_2], combination_mode="xor")

In [ ]:
plg_export = specviz.plugins["Export"]

In [ ]:
plg_export.subset = "Subset 1"

In [ ]:
plg_export.subset_format = "ecsv"

In [ ]:
plg_export.export(overwrite=True)  # Subset_1.ecsv

In [ ]:
!cat Subset_1.ecsv

In [ ]:
specviz_2 = Specviz()

In [ ]:
specviz_2.load_data(sp, data_label="Spectrum")

In [ ]:
specviz_2.show()

In [ ]:
plg_subset_2 = specviz_2.plugins["Subset Tools"]

In [ ]:
# Because ECSV output was simplified, the combination_mode can break
# things if you are not careful!
#plg_subset_2.import_region("Subset_1.ecsv")  # BAD -- first subregion missing
plg_subset_2.import_region("Subset_1.ecsv", combination_mode="or")  # OK
#plg_subset_2.import_region("Subset_1.ecsv", combination_mode="and")  # BAD -- null subset
#plg_subset_2.import_region("Subset_1.ecsv", combination_mode="new")  # BAD -- two subsets

### Additional work

If we want perfect roundtrip, to do it properly, we need an equivalent of
[compound region](https://astropy-regions.readthedocs.io/en/latest/compound.html) for
[SpectralRegion](https://specutils.readthedocs.io/en/stable/spectral_regions.html) upstream.
This could take significant effort, so we might have to just acknowledge the limitations
and move on. Is keeping original inputs that important if simplified version still give
you the same thing?

If you want support for arbitrary operators from the same ECSV file, this is not possible
without native operator support in `specutils` (see above).

The `import_region` API can have "wild" behaviors depending on `combination_mode`
when a filename is given. This needs to be reined in.

### Recommendations

1. Acknowledge that original inputs might be lost upon saving it back out
   because we simplify the composite spectral regions.
2. Acknowledge that one file can only store one subset.
   (Needs extra pondering if you also want to support `"new"` to create one subset
   per file entry.)
4. Only allow `combination_mode="or"` when a filename is given.
   Since default is actually `None` (which `glue` translated to `"replace"`,
   maybe have to recode `None` to mean `"or"` when input is filename.
   Raise exception for any other value.